In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df_soil = pd.read_csv("datasets/drought/soil_data.csv")
df_test = pd.read_csv("datasets/drought/test_timeseries/test_timeseries.csv")

In [ ]:
df_soil.columns

In [ ]:
df_test.columns

In [ ]:
df_soil
df_soil['fips'].nunique()

In [ ]:
df_soil = df_soil[["fips", "lat", "lon", "elevation"]]
df_soil

In [ ]:
import numpy as np

In [ ]:
df_test[["lat", "lon", "elevation"]] = np.nan
df_test

In [ ]:
for index, row in df_soil.iterrows():
    fips = row["fips"]
    lat = row["lat"]
    lon = row["lon"]
    elev = row["elevation"]
    df_test.loc[(df_test.fips == fips),'lat'] = lat
    df_test.loc[(df_test.fips == fips),'lon'] = lon
    df_test.loc[(df_test.fips == fips),'elevation'] = elev
    print(index)
df_test

In [ ]:
df_test.to_csv("datasets/drought/updated_test_timeseries.csv")

In [ ]:
df_test['score'].nunique()

In [ ]:
df2 = df_test[~df_test['score'].isnull()]
df2

In [ ]:
df2.columns

In [ ]:
# 'fips', 'date', 
df3 = df2[['PRECTOT', 'PS', 'QV2M', 'T2M', 'T2MDEW', 'T2MWET', 'T2M_MAX', 'T2M_MIN', 'T2M_RANGE', 'score']]
df4 = df2[['TS', 'WS10M', 'WS10M_MAX', 'WS10M_MIN', 'WS10M_RANGE', 'WS50M', 'WS50M_MAX', 'WS50M_MIN', 'score']]
df5 = df2[['WS50M_RANGE', 'lat', 'lon', 'elevation', 'score']]

In [ ]:
cor = df3.corr()
sns.heatmap(cor, annot=True)
plt.show()

In [ ]:
cor = df4.corr()
sns.heatmap(cor, annot=True)
plt.show()

In [ ]:
cor = df5.corr()
sns.heatmap(cor, annot=True)
plt.show()

We'll use PS (surface pressure), lon (longitude), elevation as features

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

In [ ]:
X = df2[["PS", "T2M", "lon", "elevation"]]
y = df2[["score"]]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.50)

In [ ]:
lin_regressor = LinearRegression()
lin_regressor.fit(X_train, y_train)

In [ ]:
y_pred = lin_regressor.predict(X_test)
y_pred

In [ ]:
r2_score(y_test, y_pred)

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

In [ ]:
poly_reg = PolynomialFeatures(degree=3)
X_poly = poly_reg.fit_transform(X_train)
poly_reg.fit(X_poly, y_train)
lin_reg2 = LinearRegression()
lin_reg2.fit(X_poly, y_train)
y_pred2 = lin_regressor.predict(X_test)
r2_score(y_test, y_pred2)
